The general outline of the algorithm is to:

1-Compute the Scharr gradient magnitude representations in both the x and y direction.<br>
2-Subtract the y-gradient from the x-gradient to reveal the barcoded region.<br>
3-Blur and threshold the image.<br>
4-Apply a closing kernel to the thresholded image.<br>
5-Perform a series of dilations and erosions.<br>
6-Find the largest contour in the image, which is now presumably the barcode.<br>

In [9]:
# USAGE
# python detect_barcode.py --image images/barcode_01.jpg

# import the necessary packages
import numpy as np
import argparse
import imutils
import cv2


# load the image and convert it to grayscale
image = cv2.imread("images/nav1.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("image",image)
cv2.imshow("gray",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [10]:
# compute the Scharr gradient magnitude representation of the images
# in both the x and y direction using OpenCV 2.4
ddepth = cv2.cv.CV_32F if imutils.is_cv2() else cv2.CV_32F

# we use the Scharr operator (specified using ksize = -1 ) to construct the gradient magnitude representation of the 
#grayscale image in the horizontal and vertical directions on
gradX = cv2.Sobel(gray, ddepth=ddepth, dx=1, dy=0, ksize=-1)
gradY = cv2.Sobel(gray, ddepth=ddepth, dx=0, dy=1, ksize=-1)

# subtract the y-gradient from the x-gradient
'''
From there, we subtract the y-gradient of the Scharr operator from the x-gradient of the Scharr operator.
By performing this subtraction we are left with regions of the image that have high horizontal gradients 
and low vertical gradients
'''
gradient = cv2.subtract(gradX, gradY)

gradient = cv2.convertScaleAbs(gradient)

cv2.imshow("gray",gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()



# filter out the noise in the image and focus solely on the barcode region.

In [11]:
# blur and threshold the image

# smooth out high frequency noise in the gradient representation of the image.
blurred = cv2.blur(gradient, (9, 9))
#Any pixel in the gradient image that is not greater than 225 is set to 0 (black). Otherwise, the pixel is set to 255 (white).
(_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)

cv2.imshow("thresh",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()



<b>close these gaps </b>and make it easier for our algorithm to detect the “blob”-like region of the barcode, we’ll need to perform some basic morphological operations:

In [12]:
# construct a closing kernel and apply it to the thresholded image

#This kernel has a width that is larger than the height, thus allowing us to close the gaps between vertical stripes of the barcode.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
#applying kernal to the image
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
cv2.imshow("close",closed)
cv2.waitKey(0)
cv2.destroyAllWindows()

we have some <b>other blobs in image that are irrelevannt </b>not a part of barcode so remove them

In [13]:
'''
An erosion will “erode” the white pixels in the image, thus removing the small blobs, 
whereas a dilation will “dilate” the remaining white pixels and grow the white regions back out.
'''
# perform a series of erosions and dilations
closed = cv2.erode(closed, None, iterations = 4)
closed = cv2.dilate(closed, None, iterations = 4)
cv2.imshow("close",closed)
cv2.waitKey(0)
cv2.destroyAllWindows()

# contours of the barcoded region of the image:

In [ ]:

# find the contours in the thresholded image, then sort the contours
# by their area, keeping only the largest one
cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]

# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = cv2.cv.BoxPoints(rect) if imutils.is_cv2() else cv2.boxPoints(rect)
box = np.int0(box)

# draw a bounding box arounded the detected barcode and display the
# image
cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()